# Anexo A

In [58]:
#Importar librerías

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px


# Replicar los cálculos de estadísticas realizados en el archivo

In [59]:
# Cargar archivo Excel

df = pd.read_excel('CR EJEMPLO MOD.xlsm', sheet_name=None)

/Users/paulinalealmosqueda/anaconda3/envs/IDM/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed

/Users/paulinalealmosqueda/anaconda3/envs/IDM/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning:

Data Validation extension is not supported and will be removed



In [60]:
# Traer la hoja de budget proveniente del archivo

df_budget = pd.read_excel('CR EJEMPLO MOD.xlsm', sheet_name='Budget', header=None)
# Establecer la segunda fila (índice 1) como índice
df_budget.columns = df_budget.iloc[1]  # Esto asigna la segunda fila como nombres de columnas
df_budget = df_budget.drop([0, 1])  # Elimina las dos primeras filas (usadas como índice y encabezado)

#Hacer drop de las columnas que no se usaran en el cost report.
# Ver información del budget.

df_budget = df_budget.loc[:, df_budget.columns.notna()]
df_budget = df_budget.drop('ALL', axis = 1)
df_budget = df_budget.drop('Costo', axis = 1)


df_budget.head()

1,TIPO,REF,Etapa,Sección,ACCT,BUDGET
2,Above the Line,Cuenta General,NaN,NaN,1100,10065000
3,Above the Line,Sub cuenta,NaN,NaN,1101,8340000
4,Above the Line,Detalle,NaN,NaN,1101-002,0
5,Above the Line,Detalle,NaN,NaN,1101-004,0
6,Above the Line,Detalle,DEVELOPMENT,NaN,1101-005,1400000


In [61]:
# Tomar las hojas de Data y de Comprometidos

data = df['Data']
comprometidos = df['Comprometidos']


# Renombrar columna de ACCT como Partida presupuestal. Agrupar según el subtotal
data = data.rename(columns = {"PARTIDA PRESUPUESTAL": "ACCT"})
subTotal = data.groupby("ACCT")["SUBTOTAL"].sum()
subTotal = subTotal.reset_index()

In [62]:
# Hacer un merge con outer para no perder valores y crear dataframe de CostReport.

CostReport = pd.merge(df_budget, subTotal,on="ACCT",how = "outer")
CostReport = CostReport.rename(columns = {"SUBTOTAL": "COST TO DATE"})

In [63]:
# Eliminar las columnas de la hoja de comprometidos que no se usarán.

comprometidos = comprometidos.drop('CONCEPTO', axis = 1)
comprometidos = comprometidos.drop('MONTO', axis = 1)
comprometidos = comprometidos.drop('PAGO', axis = 1)


In [64]:
#Cambiar el nombre de comprometidos, para poder hacer un merge adecuado.
#Juntar los valores de comprometidos con el CostReport.

comprometidos = comprometidos.rename(columns = {"CUENTA": "ACCT"})

CostReport = pd.merge(CostReport, comprometidos,on="ACCT",how = "outer")
CostReport = CostReport.rename(columns = {"NVO MONTO": "COMMITMENTS"})
CostReport = CostReport.loc[CostReport['REF'].isin(['Detalle'])] # Filtrar solo para subcuentas en Detalle.

# Hacer que el presupuesto y Cost to Date no tenga valores Nan.
CostReport['BUDGET'] = CostReport['BUDGET'].fillna(0)
CostReport['COST TO DATE'] = CostReport['COST TO DATE'].fillna(0)


/var/folders/4z/x0x3cnw55vgc_j0fssxl_9fm0000gn/T/ipykernel_8094/3791871464.py:11: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [65]:
# Cálculos para el resto de las columnas

#Costo total = Cost to date + Commitments

total_ctd = CostReport['COST TO DATE'].fillna(0) + CostReport['COMMITMENTS'].fillna(0)
CostReport['TOTAL CTD'] = total_ctd

# Fórmula que normalmente se usaría:
#estimate_complete = abs(CostReport['BUDGET'].fillna(0)) - CostReport['TOTAL CTD'].fillna(0)

estimate_complete = 0

#Poner todos los valores como cero.
CostReport['ESTIMATE TO COMPLETE (ETC)'] = estimate_complete


#Estimar costo final = total + estimado
est_final_cost = total_ctd + estimate_complete


CostReport['ESTIMATE FINAL COST (EFC)'] = est_final_cost


# Varianza = presupuesto - estimado final
variance = (CostReport['BUDGET']) - CostReport['ESTIMATE FINAL COST (EFC)']


CostReport['VARIANCE $'] = variance

CostReport['P ESTIMATE FINAL COST'] = variance


# Agregar subcuentas faltantes.
Este proceso se realizo de manera manual, al ver las subcuentas que en el Excel no tenían asignado Post ni All Show en la hoja de budget, pero si en Cost Report.

In [66]:
# Cargar el archivo Excel
file_path = 'CR EJEMPLO MOD.xlsm'

# Leer la hoja "Cost Report" sin encabezados iniciales
try:
	cost_report_df = pd.read_excel(file_path, sheet_name="Cost Report", header=None)
except PermissionError:
	print(f"Permission denied: Please close '{file_path}' in all other programs and try again.")
	raise

# Eliminar las primeras 8 filas y las primeras 2 columnas
data_cleaned = cost_report_df.iloc[8:, 2:]

# Asignar la primera fila como encabezados
data_cleaned.columns = data_cleaned.iloc[0]
data_cleaned = data_cleaned[1:]

# Resetear el índice
data_cleaned.reset_index(drop=True, inplace=True)

# Mostrar las primeras filas de la tabla limpia
data_cleaned.head()

# Eliminar columnas con encabezados NaN o vacíos
data_cleaned = data_cleaned.loc[:, ~data_cleaned.columns.isna()]

# Mostrar las primeras filas de la tabla limpia después de eliminar columnas con encabezado NaN
print(data_cleaned.head())

# Guardar la tabla limpia en un nuevo archivo Excel (opcional)
data_cleaned.to_excel("Cost_Report_Limpio_Sin_NA.xlsx", index=False)

crep = pd.read_excel('Cost_Report_Limpio_Sin_NA.xlsx')

8            TIPO             REF        Etapa Sección      ACCT  1.0  \
0  Above the Line  Cuenta General          NaN     NaN      1100  NaN   
1  Above the Line      Sub cuenta          NaN     NaN      1101  NaN   
2  Above the Line         Detalle          NaN     NaN  1101-002  NaN   
3  Above the Line         Detalle          NaN     NaN  1101-004  NaN   
4  Above the Line         Detalle  DEVELOPMENT     NaN  1101-005  NaN   

8 COST TO DATE COMMITMENTS    TOTAL CTD ESTIMATE TO COMPLETE (ETC)  \
0  10065000.08           0  10065000.08                          0   
1   8340000.02           0   8340000.02                          0   
2            0           0            0                          0   
3            0           0            0                          0   
4      1400000           0      1400000                          0   

8 ESTIMATE FINAL COST (EFC)    BUDGET VARIANCE $ INDEX  2.0  \
0               10065000.08  10065000      -0.08   NaN  NaN   
1             

In [67]:
#Agregar valor faltante de Post

missing_post = CostReport[CostReport['ACCT'] == '5221-001']
CostReport.at[2258,'Etapa']='POST'
CostReport[CostReport['ACCT'] == '5221-001']


# Cuentas manualmente encontradas, donde la etapa era ALL SHOW y no estaba siendo registrada

CostReport[CostReport['ACCT'] == '1621-002']
CostReport[CostReport['ACCT'] == '3193-004']
CostReport[CostReport['ACCT'] == '3902-047']
CostReport[CostReport['ACCT'] == '7201-002']
CostReport[CostReport['ACCT'] == '7230-001']
CostReport[CostReport['ACCT'] == '7230-002']
CostReport[CostReport['ACCT'] == '7230-003']
CostReport[CostReport['ACCT'] == '7230-004']
CostReport[CostReport['ACCT'] == '7230-005']

# Agregar valores faltantes para ALL SHOW
missing_show = CostReport[CostReport['ACCT'] == '1621-002']
CostReport.at[448,'Etapa']='ALL SHOW'
CostReport.at[1420,'Etapa']='ALL SHOW'
CostReport.at[2043,'Etapa']='ALL SHOW'
CostReport.at[2460	,'Etapa']='ALL SHOW'
CostReport.at[2473	,'Etapa']='ALL SHOW'
CostReport.at[2474	,'Etapa']='ALL SHOW'
CostReport.at[2475	,'Etapa']='ALL SHOW'
CostReport.at[2476	,'Etapa']='ALL SHOW'
CostReport.at[2477	,'Etapa']='ALL SHOW'

CostReport[CostReport['ACCT'] == '7230-005']

,TIPO,REF,Etapa,Sección,ACCT,BUDGET,COST TO DATE,COMMITMENTS,TOTAL CTD,ESTIMATE TO COMPLETE (ETC),ESTIMATE FINAL COST (EFC),VARIANCE $,P ESTIMATE FINAL COST
2477,Others,Detalle,ALL SHOW,NaN,7230-005,0.0,1320.29,NaN,1320.29,0,1320.29,-1320.29,-1320.29


**Cambiar etiquetas de Etapa, en caso de que no sean etapas.**

Fringes, COVID, ALL SHOW y 2nd Unit, no son etapas. Estas se deberán de asignar a la etapa de SHOOT.

In [68]:
# Filtrar el DataFrame para incluir solo las filas donde REF es "Detalle"
CostReport_ref = CostReport

# Reemplazar valores en la columna 'Etapa'
CostReport_ref['Etapa'] = CostReport_ref['Etapa'].replace({
    'Fringes': 'SHOOT',
    'COVID': 'SHOOT',
    'ALL SHOW': 'SHOOT',
    '2nd Unit': 'SHOOT'
})

# Crear columna CUENTA con primeros 4 dígitos de ACCT
CostReport_ref['CUENTA'] = CostReport_ref['ACCT'].astype(str).str[:4]
CostReport_ref


,TIPO,REF,Etapa,Sección,ACCT,BUDGET,COST TO DATE,COMMITMENTS,TOTAL CTD,ESTIMATE TO COMPLETE (ETC),ESTIMATE FINAL COST (EFC),VARIANCE $,P ESTIMATE FINAL COST,CUENTA
4,Above the Line,Detalle,NaN,NaN,1101-002,0.0,0.000,NaN,0.000,0,0.000,0.000,0.000,1101
5,Above the Line,Detalle,NaN,NaN,1101-004,0.0,0.000,NaN,0.000,0,0.000,0.000,0.000,1101
6,Above the Line,Detalle,DEVELOPMENT,NaN,1101-005,1400000.0,1400000.000,NaN,1400000.000,0,1400000.000,0.000,0.000,1101
7,Above the Line,Detalle,DEVELOPMENT,NaN,1101-006,100000.0,100000.030,NaN,100000.030,0,100000.030,-0.030,-0.030,1101
8,Above the Line,Detalle,DEVELOPMENT,NaN,1101-007,100000.0,100000.000,NaN,100000.000,0,100000.000,0.000,0.000,1101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,Others,Detalle,SHOOT,NaN,7230-001,0.0,7050.755,NaN,7050.755,0,7050.755,-7050.755,-7050.755,7230
2474,Others,Detalle,SHOOT,NaN,7230-002,0.0,11436.910,NaN,11436.910,0,11436.910,-11436.910,-11436.910,7230
2475,Others,Detalle,SHOOT,NaN,7230-003,0.0,12178.920,NaN,12178.920,0,12178.920,-12178.920,-12178.920,7230
2476,Others,Detalle,SHOOT,NaN,7230-004,0.0,3025.870,NaN,3025.870,0,3025.870,-3025.870,-3025.870,7230


Se verifica que la cantidad de etapas sea igual a las del Excel.

In [69]:
CostReport_ref['Etapa'].value_counts()

Etapa
SHOOT          1063
PREP            212
POST            147
WRAP            106
DEVELOPMENT      29
SOFT             16
Name: count, dtype: int64

In [70]:
len(CostReport_ref[CostReport_ref['BUDGET']==0])

871

Los valores coinciden con los que CostReport tiene en el Excel. Se han replicado las estadísticas realizadas en el archivo. El siguiente paso consiste en explorar la distribución del presupuesto, cost to date y la varianza para generar una hipótesis.

# Exploración

In [71]:
CostReport_ref.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2234 entries, 4 to 2477
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TIPO                        2234 non-null   object 
 1   REF                         2234 non-null   object 
 2   Etapa                       1573 non-null   object 
 3   Sección                     0 non-null      object 
 4   ACCT                        2234 non-null   object 
 5   BUDGET                      2234 non-null   float64
 6   COST TO DATE                2234 non-null   float64
 7   COMMITMENTS                 9 non-null      float64
 8   TOTAL CTD                   2234 non-null   float64
 9   ESTIMATE TO COMPLETE (ETC)  2234 non-null   int64  
 10  ESTIMATE FINAL COST (EFC)   2234 non-null   float64
 11  VARIANCE $                  2234 non-null   float64
 12  P ESTIMATE FINAL COST       2234 non-null   float64
 13  CUENTA                      2234 non-n

In [72]:
CostReport_ref['BUDGET'].describe()

count    2.234000e+03
mean     6.047355e+04
std      3.054874e+05
min     -8.495155e+05
25%      0.000000e+00
50%      7.000000e+03
75%      4.400000e+04
max      1.164357e+07
Name: BUDGET, dtype: float64

In [73]:
CostReport_ref['BUDGET'].sum()

np.float64(135097911.0)

El presupuesto total fue de 135097911, el cual se distribuyó de manera diferente dependiendo de la subcuenta. Se encontró que hay subcuentas con presupuesto negativo y hay otras que no tienen.

In [74]:
CostReport_ref['COST TO DATE'].describe()

count    2.234000e+03
mean     6.014860e+04
std      2.882579e+05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      4.000000e+04
max      1.057418e+07
Name: COST TO DATE, dtype: float64

In [75]:
CostReport_ref['COST TO DATE'].sum()

np.float64(134371982.764)

El cost to date total fue de 134371982.764. Se observa que hay cuentas que no gastaron, lo cual podría significar que no todas las cuentas aprovecharon el presupuesto que se les dio.

In [76]:
CostReport_ref['VARIANCE $'].describe()

count    2.234000e+03
mean    -1.556461e+02
std      7.661060e+04
min     -8.495155e+05
25%      0.000000e+00
50%      0.000000e+00
75%      1.419933e+03
max      2.699345e+06
Name: VARIANCE $, dtype: float64

In [77]:
CostReport_ref['VARIANCE $'].sum()

np.float64(-347713.41400000034)

In [78]:
CostReport_ref[CostReport_ref['VARIANCE $'] == 0]

,TIPO,REF,Etapa,Sección,ACCT,BUDGET,COST TO DATE,COMMITMENTS,TOTAL CTD,ESTIMATE TO COMPLETE (ETC),ESTIMATE FINAL COST (EFC),VARIANCE $,P ESTIMATE FINAL COST,CUENTA
4,Above the Line,Detalle,NaN,NaN,1101-002,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,1101
5,Above the Line,Detalle,NaN,NaN,1101-004,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,1101
6,Above the Line,Detalle,DEVELOPMENT,NaN,1101-005,1400000.0,1400000.0,NaN,1400000.0,0,1400000.0,0.0,0.0,1101
8,Above the Line,Detalle,DEVELOPMENT,NaN,1101-007,100000.0,100000.0,NaN,100000.0,0,100000.0,0.0,0.0,1101
9,Above the Line,Detalle,NaN,NaN,1101-010,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,1101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,Others,Detalle,SHOOT,NaN,7201-002,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,7201
2462,Others,Detalle,NaN,NaN,7201-006,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,7201
2464,Others,Detalle,SHOOT,NaN,7201-008,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,7201
2466,Others,Detalle,NaN,NaN,7216-002,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,7216


In [79]:
CostReport_ref['VARIANCE $'].info()

<class 'pandas.core.series.Series'>
Index: 2234 entries, 4 to 2477
Series name: VARIANCE $
Non-Null Count  Dtype  
--------------  -----  
2234 non-null   float64
dtypes: float64(1)
memory usage: 99.4 KB


La varianza total fue de -347713.41, lo que significa que el proyecto se excedió de lo presupuestado. A partir de estas observaciones, se puede intuir que el aprovechamiento del presupuesto es diferente dependiendo de la subcuenta. Por lo tanto, se plantea la siguiente hipótesis. 

# Hipótesis
El aprovechamiento del presupuesto asignado a cada cuenta influye directamente en sí se estaba generando ahorro o, por el contrario, provocando sobrecostes.  

# Dearrollo

Primero, se creará una función para resumir los resultados dependiendo del tipo de cuenta.

In [80]:
# Crear una función para el resumen

def summary_etapas(CostReport, etapa):
    
    #Hacer un filtro por etapas
  etapa_filtro = etapa
  colores = [
      "#0B5566", "#21B4B4"]
  
  # Si el filtro es ALL, toma todo el CostReport; si no, filtra por etapa
  if etapa_filtro == "ALL":
      df_filtrado = CostReport.copy()
  else:
      df_filtrado = CostReport[CostReport['Etapa'] == etapa_filtro].copy()

  # Agrupa por TIPO
  summary = df_filtrado.groupby('TIPO').agg({
      'COST TO DATE': 'sum',
      'COMMITMENTS': 'sum',
      'BUDGET': 'sum',
      'ESTIMATE TO COMPLETE (ETC)': 'sum'
  }).reset_index()
  
  # Replicar cálculos para el summary
  summary['TOTAL CTD'] = summary['COST TO DATE'] + summary['COMMITMENTS']
  summary['ESTIMATE FINAL COST (EFC)'] = summary['TOTAL CTD'] + summary['ESTIMATE TO COMPLETE (ETC)']
  summary['VARIANCE $'] = summary['BUDGET'] - summary['ESTIMATE FINAL COST (EFC)']
  summary['INDEX'] =  (summary['ESTIMATE FINAL COST (EFC)']/ summary['BUDGET'] ) * 100

  # Agrega el total general. Generar una base de datos con todas las columnas.
  total_row = pd.DataFrame({
      'TIPO': ['TOTAL'],
      'COST TO DATE': [summary['COST TO DATE'].sum()],
      'COMMITMENTS': [summary['COMMITMENTS'].sum()],
      'BUDGET': [summary['BUDGET'].sum()],
      'ESTIMATE TO COMPLETE (ETC)': [summary['ESTIMATE TO COMPLETE (ETC)'].sum()],
      'TOTAL CTD': [summary['TOTAL CTD'].sum()],
      'ESTIMATE FINAL COST (EFC)': [summary['ESTIMATE FINAL COST (EFC)'].sum()],
      'VARIANCE $': [summary['VARIANCE $'].sum()],
      'INDEX': [summary['ESTIMATE FINAL COST (EFC)'].sum() / summary['BUDGET'].sum() *100]

  })

  summary = pd.concat([summary, total_row], ignore_index=True)


  # Reordena columnas
  summary = summary[['TIPO', 'COST TO DATE', 'COMMITMENTS', 'TOTAL CTD',
                    'ESTIMATE TO COMPLETE (ETC)', 'ESTIMATE FINAL COST (EFC)',
                    'BUDGET', 'VARIANCE $', 'INDEX']]


  #Desplegar resumen como se ve en el Excel.
  display(summary.round())
  
  # Preparar información para visualizar.
  metrics = ["COST TO DATE","COMMITMENTS","TOTAL CTD","ESTIMATE TO COMPLETE (ETC)","ESTIMATE FINAL COST (EFC)","VARIANCE $"]
  rows = 2
  columns = 2
  TIPOS = summary[summary["TIPO"] != "TOTAL"]["TIPO"].tolist()
  metric_labels = {
        "COST TO DATE": "COST TO DATE",
        "COMMITMENTS": "COMMITMENTS",
        "TOTAL CTD": "TOTAL COST TO DATE",
        "ESTIMATE TO COMPLETE (ETC)": "ESTIMATE TO COMPLETE",
        "ESTIMATE FINAL COST (EFC)": "ESTIMATE FINAL COST",
        "VARIANCE $": "VARIANCE"
    }
  
  # Poner colores para cada métrica.
  color_by_metric = {
        "COST TO DATE": "royalblue",
        "COMMITMENTS": "orange",
        "TOTAL CTD": "green",
        "ESTIMATE TO COMPLETE (ETC)": "purple",
        "ESTIMATE FINAL COST (EFC)": "teal",
        "VARIANCE $": "brown",
    }

    # Hacer gráfica estilo gauge y agregar botones interactivos.

  fig = go.Figure()

  for metric in metrics:
        visible = (metric == "TOTAL CTD")
        for i, (_, row) in enumerate(summary[summary["TIPO"] != "TOTAL"].iterrows()):
            fig.add_trace(go.Indicator(
                mode="gauge+number",
                value=row[metric],
                number={
                    'valueformat': "$,.0f",
                    'font': {'size': 30}
                },
                #Calcular porcentaje
                title={
                    'text': f"<b>{row['TIPO']}</b><br>{(row[metric] / row['BUDGET']) * 100:.1f}%",
                    'font': {'size': 20}
                },
                gauge={
                    'axis': {'range': [0, row["BUDGET"] * 1.2]},
                    'bar': {'color': color_by_metric.get(metric, "gray")},
                    'bgcolor': "lightgray",
                    'threshold': {
                        'line': {'color': "red", 'width': 3},
                        'thickness': 0.75,
                        'value': row["BUDGET"]
                    }
                },
                domain={'row': i // columns, 'column': i % columns},
                visible=visible
            ))
  buttons = []
  for i, metric in enumerate(metrics):
        visibility = [False] * len(metrics) * len(TIPOS)
        for j in range(len(TIPOS)):
            visibility[i * len(TIPOS) + j] = True
        buttons.append(dict(
            label=metric_labels.get(metric, metric),
            method="update",
            args=[
                {"visible": visibility},
                {
                    "title": {
                        "text": f"Summary: {metric_labels.get(metric, metric)}",
                        "font": {"size": 24},
                        "x": 0.5,
                        "y": 0.98,
                        "xanchor": "center"
                    }
                }
            ]
        ))
  fig.update_layout(
        margin=dict(t=150),
        height=800,
        width=1000,
        grid={'rows': rows, 'columns': columns, 'pattern': "independent"},
        title={
            "text": "Summary: COST TO DATE",
            "font": {"size": 24},
            "x": 0.5,
            "y": 0.98,
            "xanchor": "center"
        },
        updatemenus=[{
            "buttons": buttons,
            "direction": "down",
            "showactive": True,
            "x": 0,
            "y": 1.25,
            "xanchor": "center",
            "yanchor": "top",
            "font": {"size": 12, "color": "black"},
            "bgcolor": "lightgray",
            "bordercolor": "gray"
        }]
    )

  fig.show()


El resumen se despliega dependiendo de los parámetros que se pongan en la función. 

Esta función debe de contener la base de datos, la cual se recomienda que sea CostReport_ref, ya que esta tiene las etapas bien establecidas.

El segundo argumento debe de ser la etapa que se desea filtrar. Cabe destacar, que el resultado en el resumen de SHOOT será diferente al que se ve en el Excel, porque esta ya incluye la suma de Fringes, COVID, ALL SHOW y 2nd Unit. 

Por otra parte, se despliaga una serie de gráficas divididas por el tipo de subcuenta (Above the Line, Below the Line, Others y Post). En esta, el usuario puede seleccionar la columna que le interesa visualizar. Las gráficas muestran el gasto en verde y el presupuesto en una línea roja, de esta manera, se puede observar si se excedió o no del presupuesto. Además de esto, se muestran los valores de manera absoluta y en porcentaje.

# Figura 1. Summary

In [81]:
summary_etapas(CostReport_ref, 'ALL')

,TIPO,COST TO DATE,COMMITMENTS,TOTAL CTD,ESTIMATE TO COMPLETE (ETC),ESTIMATE FINAL COST (EFC),BUDGET,VARIANCE $,INDEX
0,Above the Line,37179095.0,0.0,37179095.0,0,37179095.0,35701887.0,-1477207.0,104.0
1,Below the Line,68434405.0,0.0,68434405.0,0,68434405.0,69999853.0,1565448.0,98.0
2,Others,14267410.0,1073642.0,15341052.0,0,15341052.0,14989184.0,-351868.0,102.0
3,Post,14491072.0,0.0,14491072.0,0,14491072.0,14406986.0,-84086.0,101.0
4,TOTAL,134371983.0,1073642.0,135445624.0,0,135445624.0,135097911.0,-347713.0,100.0


Con base en los resúmenes se observa que hay subcuentas que se pasan del presupuesto y también hay algunas que no lo aprovechan. En este caso, las subcuentas en Above the line Others y Post, se encontraron que se sobrepasaron de su presupuesto con un porcentaje de utilización del 104.1%, 102.3% y 100.6%, respectivamente. Por otro parte, las subcuentas de Below the Line, ahorraron dinero, al utilizar el 97.8% de su presupuesto (los resultados de total cost to date y estimate final cost son iguales porque el proyecto ya terminó). Observando, la columna de commitments con el filtro, se puede ver que solo las subcuentas de Others tuvieron compremetidos y que la cantidad a deber representar el 7.2% del presupuesto total. En el caso del estimado completo, todas las subcuentas se encuentran en cero porque el proyecto ya terminó. Por otra parte, las únicas subcuentas que tuvieron varianza positiva fueron las de Below the Line, ya que se ahorró dinero. 

Este mismo análisis se puede realizar por etapa, de manera en que la empresa pueda comprender de mejor manera los movimientos que se realizaron y tener una imagen general de lo que sucede. A continuación se dara una descripción de los resultados por estaba y por columna.

Development: Las cuentas utilizaron todo el presupuesto sin excederse para Above the Line y Below the Line. En el caso de Others, unicamente se uso el 9% del presupuesto y no hubo gastos en Post. Además de esto, la etapa no tuvo cuentas compremetidas y la etapa solo tuvo una varianza positiva con Others (solo se uso el 9%) mientras que para Above the Line y Below the Line, fue de 0%, lo cual coincide con los resultados encontrados en la columna de Cost to Date.

Prep: Las subcuentas en Below the Line y Others utulizaron el 100.7% y 310.4% de su presupuesto respectivamente. Esto quiere decir que sumando las subcuentas, hubo un sobregiro. En particular, se observa que el gasto de Others estuvo 210.4% por arriba del presupuesto.  Mientras tanto, las cuentas de Above the Line y Post usaron el 77.9% y el 93.1% del presupuesto asignado, lo que significa que ahorraron. Por ende, como un plan de acción a futuro, se puede redistribuir el presupuesto para que Others no se exceda y que no hayan cuentas que desaprovechan el dinero asignado. Por otro lado, no hubieron cuentas comprometidas y las únicas cuentas con varianza positiva fueron las de Above the Line y Post, lo cual coincide con lo encontrado anteriormente.

Shoot: Las subcuentas Above the Line y Post utilizaron un 107.7% y 125.7% del presupuesto, es decir se pasaron del dinero asignado. En cambio, las subcuentas de Others y Below the Line ahorraron con una utlización del 92.9% y 97.6%. En esta estapa si hubieron cuentas comprometidas, las cuales representaron el 7.3% del presupuesto de others. Asimismo, las unicas subcuentas que tuvieron una varianza fueron las de Below the Line.

WRAP: Above the line se sobrepaso del presupuesto, Below the line unicamente utilizo el 87.2%, Others utilizó el 100% sin tener vairanza y Post no tuvo ni prespuesto ni gato. En este caso, no hubeiron cuentas comprometidas y Below the Line fue la unia subcuenta con varianza positiva, es decir que ahorraron diner.

SOFT: La unica cuenta que se paso de su presupuesto fue Below the Line con un 4.6% de sobregiro. Por otro lado, no hubieron gastos en Post ni presupuesto. Tampoco se registraron cuentas comprometidas y la unica cuenta con varianza negativa fue Below the Line, dado a que se paso del presupuesto.

Por ende, con estos se resumenes es posible entender que la distribución entre presupuesto contra gasto, varía dependiendo de la etapa, ya que los resultados entre tipo de subcuenta fueron diferentes. Por ejemplo, en el caso de la preproducción, Above the Line ahorro dinero, pero en la etapa de Shoot se exedieron. A partir de esto, se puede intuir que el comportamiento particular de cada subcuenta tambien va a variar.


Por esta razón, lo siguiente es crear gráficas para encontrar el aprovechamiento del presupuesto y cómo es que este influye en la subcuenta, ya sea si está ahorrando o si tiene sobregiros.

**Clasificar el estado de las cuentas**

Criterio de clasificación:
* Sin presupuesto pero con gasto.
* Gastó más que el presupuesto
* Ahorro
* Gastó exacto

In [82]:
# Filtrar el DataFrame para incluir solo las filas donde REF es "Detalle"
CostReport_ref = CostReport

# Reemplazar valores en la columna 'Etapa'
CostReport_ref['Etapa'] = CostReport_ref['Etapa'].replace({
    'Fringes': 'SHOOT',
    'COVID': 'SHOOT',
    'ALL SHOW': 'SHOOT',
    '2nd Unit': 'SHOOT'
})

# Crear columna CUENTA con primeros 4 dígitos de ACCT
CostReport_ref['CUENTA'] = CostReport_ref['ACCT'].astype(str).str[:4]


resumen_1 = (
    CostReport_ref
    .groupby(['CUENTA', 'TIPO', 'COMMITMENTS', 'Etapa'], dropna=False)[['COST TO DATE', 'BUDGET']]
    .sum()
    .reset_index()
)

resumen =(
    resumen_1.groupby(['CUENTA', 'TIPO'], dropna=False)[['COST TO DATE', 'BUDGET', 'COMMITMENTS']]
    .sum()
    .reset_index()
)

# Crear columna 'ESTADO' en resumen.BUDGET
# Incluir las categorías 

def clasificar_estado(row):
    if row['BUDGET'] == 0 and row['COST TO DATE'] > 0:
        return 'Sin presupuesto asignado pero con gasto'
    elif row['COST TO DATE'] > row['BUDGET']:
        return 'Gastó más que el presupuesto'
    elif row['COST TO DATE'] < row['BUDGET']:
        return 'Ahorro'
    else:
        return 'Gastó exacto el presupuesto'

resumen['ESTADO'] = resumen.apply(clasificar_estado, axis=1)

# Listas de opciones disponibles
estados_disponibles = resumen['ESTADO'].unique().tolist()
tipos_disponibles = ['TODO'] + resumen['TIPO'].unique().tolist()

**Crear función para gráficar cuentas de manera interactiva**

Se muestra un resumen financiero y una gráfica de Pie para ver el porcentaje de las subcuentas según el criterio.

In [83]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import plotly.graph_objects as go

# Opciones de etapas
etapas = ['TODO'] + sorted(resumen['TIPO'].dropna().unique())
selector_etapa = widgets.Dropdown(
    options=etapas,
    value='TODO',
    description='Etapa:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='350px')
)

def mostrar_grafica(etapa):
    # Filtra por etapa o toma todo
    if etapa == 'TODO':
        df = resumen.copy()
    else:
        df = resumen[resumen['TIPO'] == etapa].copy()
    
    # Recalcula conteos y porcentajes
    conteo_clasificacion = df['ESTADO'].value_counts()
    total_cuentas = conteo_clasificacion.sum()
    porcentajes = (conteo_clasificacion / total_cuentas * 100).round(1)
    colores = {
        'Sin presupuesto asignado pero con gasto': '#f7b7a3',
        'Con presupuesto asignado pero sin gasto': "#5b0002",
        'Gastó más que el presupuesto': '#ff6b6b',
        'Gastó exacto el presupuesto': '#4ecdc4',
        'Ahorro': '#34c513',
    }
    etiquetas = [f"{cat}<br>{count} cuentas<br>{porcentajes[cat]}%"
                 for cat, count in conteo_clasificacion.items()]
    fig = go.Figure(data=[go.Pie(
        labels=conteo_clasificacion.index,
        values=conteo_clasificacion.values,
        text=etiquetas,
        hoverinfo='text',
        textinfo='percent',
        textfont_size=28,
        marker=dict(colors=[colores.get(cat, "#081221") for cat in conteo_clasificacion.index]),
        pull=[0.009] * len(conteo_clasificacion),
        hole=0.3
    )])
    fig.update_layout(
        title={
            'text': f'Distribución de Subcuentas por Estado<br><sup>TIPO: {etapa if etapa != "TODO" else "TODAS"}</sup>',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 35, 'color': "#000000", 'family': 'Berlin Sans FB'}
        },
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5,
            font=dict(size=24)
        ),
        margin=dict(t=100, b=100, l=20, r=20),
        paper_bgcolor="#ffffff",
        plot_bgcolor='rgba(0,0,0,0)',
        height=900,
        width=2000,
        annotations=[
            dict(
                text=f"Total: {total_cuentas} subcuentas",
                x=0.5,
                y=0.5,
                font_size=16,
                showarrow=False
            )
        ]
    )
    
    # Resumen financiero
    presupuesto_total = df['BUDGET'].sum()
    gasto_total = df['COST TO DATE'].sum() + df['COMMITMENTS'].sum()
    ahorro = presupuesto_total - gasto_total
    porcentaje_ejecucion = (gasto_total / presupuesto_total * 100) if presupuesto_total > 0 else 0
    porcentaje_ahorro = (ahorro / presupuesto_total * 100) if presupuesto_total > 0 else 0

    resumen1_md = f"""
# 📊 RESUMEN FINANCIERO
- ## **Presupuesto total asignado:** **${presupuesto_total:,.0f}**
- ## **Porcentaje de Sobregiro:** **{-(porcentaje_ahorro):.2f}%**
- ## **Dinero no presupuestado:** **${-(ahorro):,.0f}**
- ## **Porcentaje de ejecución:** **{(porcentaje_ejecucion):.2f}%**
- ## **Gasto total ejecutado:** **${gasto_total:,.0f}**
"""

    resumen_md = f"""
# 📊 RESUMEN FINANCIERO

- ## **Presupuesto total asignado:** **${presupuesto_total:,.0f}**
- ## **Porcentaje de ahorro:** **{porcentaje_ahorro:.2f}%**
- ## **Ahorro:** **${ahorro:,.0f}**
- ## **Porcentaje de ejecución:** **{porcentaje_ejecucion:.2f}%**
- ## **Gasto total ejecutado:** **${gasto_total:,.0f}**
"""

    # PRIMERO el Markdown, LUEGO la gráfica
    if porcentaje_ahorro < 0:
        display(Markdown(resumen1_md))
    else:
        display(Markdown(resumen_md))
    fig.show()

# Figura 3 - Gráfica de Distribución por Subcuentas.

In [84]:
widgets.interact(mostrar_grafica, etapa=selector_etapa)

interactive(children=(Dropdown(description='Etapa:', layout=Layout(width='350px'), options=('TODO', 'Above the…

<function __main__.mostrar_grafica(etapa)>

Como se mostró durante la exploración, la empresa gastó más de lo que tenía presupuestado con un sobregiro del 26% con un gasto total de $37,179,095. Por otro lado, para Above the Line hubo un sobregiro del 4.14%, donde 37.8% de las subcuentas se sobrepasaron. Mienras que 10.8% de ellas gastaron exactamente la cantidad del presupuesto y el 51.4% ahorraron dinero. En el caso de Below the Line, hubo un ahorro del 2.24%, donde el 42.9% de las subcuentas ahorraron, en comparación con el 45.2%. El el caso de Others y Post, tambien se optuvo un sobregiro del 2.35% y 0.58%, respectivamente. Por lo tanto, el proyecto unicamente ahorro con Below the Line, mientra que en los otros tipos de cuentas, hubo un sobregiro. 

De manera general, esto ayuda a entender el porcentaje de subcuentas dependiendo de la relación costo-presupuesto. No obstante, teniendo una idea general de como del estado (sobregiro, ahorro, uso exacto) de cada una de las subcuentas por tipo, lo siguiente es observar esta relación pero de manera particular. Para ello se harán dos figuras en donde se podrá filtrar por el tipo de subcuenta y tambien por estado en el que finalizo. Por otro lado, uno de los objetivos es encontrar las cuentas que tuvieron más impacto en el presupuesto general del proyecto. Esto ayudara a ver la importancia de cada subcuenta y por ende, se podran observar las subcuentas que influyen directamente con el presupuesto general.

Finalmente, se da una vista general de la distribución por tipo de subcuenta al observar la relación entre cuentas que se sobrepasaron y cuentas que ahorraron.


In [85]:

# Suponiendo que estos son los valores únicos de tu DataFrame 'resumen'
estados_disponibles = resumen['ESTADO'].unique().tolist()
tipos_disponibles = ['TODO'] + resumen['TIPO'].unique().tolist()

# Creas los selectores
selector_estado = widgets.Dropdown(
    options=estados_disponibles,
    description='Estado:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

selector_tipo = widgets.Dropdown(
    options=tipos_disponibles,
    description='Tipo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)


def mostrar_graficas(estado, tipo):
    # ==================== PRIMERA GRÁFICA (Top 15 Cuentas) ====================
    # Filtrar las cuentas por el estado seleccionado
    filtro = resumen[resumen['ESTADO'] == estado].copy()

    # Filtrar por tipo si no es 'TODO'
    if tipo != 'TODO':
        filtro = filtro[filtro['TIPO'] == tipo]

    if filtro.empty:
        print(f"No hay cuentas para la combinación: {estado} - {tipo}")
        return

    # Calcular diferencias y porcentajes
    filtro['DIFERENCIA_ABSOLUTA'] = filtro['COST TO DATE'] - filtro['BUDGET']
    filtro['DIFERENCIA_ABSOLUTA'] = filtro['DIFERENCIA_ABSOLUTA'].where(filtro['BUDGET'] > 0, 0)
    filtro['DIFERENCIA_PORCENTAJE_TOTAL'] = filtro['DIFERENCIA_ABSOLUTA'] / resumen['BUDGET'].sum() * 100
    filtro['DIFERENCIA_PORCENTAJE_CUENTA'] = (filtro['DIFERENCIA_ABSOLUTA'] / filtro['BUDGET']) * 100

    # Ordenar y tomar top 15
    if estado == 'Gastó más que el presupuesto':
        top = filtro.nlargest(15, 'DIFERENCIA_ABSOLUTA')
        y = top['DIFERENCIA_PORCENTAJE_CUENTA']
        y_title = 'Sobregiro (%)'
        color = '#ff6b6b'
        hover_label = 'Sobregiro: %{y:.1f}%<br>'
        text_labels = [f"{valor:.1f}%" for valor in y]
        yaxis_tickformat = '.1f'
        yaxis_ticksuffix = '%'
    elif estado == 'Ahorro':
        top = filtro.nsmallest(15, 'DIFERENCIA_ABSOLUTA')
        y = -top['DIFERENCIA_PORCENTAJE_CUENTA']  # Mostrar ahorro como positivo
        y_title = 'Ahorro (%)'
        color = '#34c513'
        hover_label = 'Ahorro: %{y:.1f}%<br>'
        text_labels = [f"{valor:.1f}%" for valor in y]
        yaxis_tickformat = '.1f'
        yaxis_ticksuffix = '%'
    else:  # Gastó exacto el presupuesto
        top = filtro.head(15)
        y = top['BUDGET']  # <--- Aquí mostramos el presupuesto
        y_title = 'Presupuesto ($)'
        color = '#4ecdc4'
        hover_label = 'Presupuesto: $%{y:,.2f}<br>'
        text_labels = [f"${valor:,.2f}" for valor in y]
        yaxis_tickformat = ',.2f'
        yaxis_ticksuffix = ''

    # Crear título dinámico
    titulo_tipo = f" - {tipo}" if tipo != 'TODO' else " - Todos los Tipos"

    fig1 = go.Figure()
    fig1.add_trace(go.Bar(
        x=top['CUENTA'],
        y=y,
        name=estado,
        marker_color=color,
        customdata=list(zip(
            top['COST TO DATE'],
            top['BUDGET'],
            top['DIFERENCIA_PORCENTAJE_CUENTA'],
            top['DIFERENCIA_PORCENTAJE_TOTAL'],
            top['TIPO']
        )),
        hovertemplate='<b>Cuenta: %{x}</b><br>' +
                      'Tipo: %{customdata[4]}<br>' +
                      'Gasto Real: $%{customdata[0]:,.2f}<br>' +
                      'Presupuesto: $%{customdata[1]:,.2f}<br>' +
                      hover_label +
                      'Diferencia vs Presupuesto Total: %{customdata[3]:.2f}%<br>' +
                      '<extra></extra>'
    ))

    fig1.update_layout(
        title={
            'text': f"Top 15 Cuentas - {estado}{titulo_tipo}",
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 29, 'color': '#333333', 'family': 'Berlin Sans FB'}
        },
        xaxis=dict(
            title='Cuentas',
            title_font=dict(size=24),
            tickangle=45,
            tickfont=dict(size=14),
            showgrid=False
        ),
        yaxis=dict(
            title=y_title,
            title_font=dict(size=20),
            tickformat=yaxis_tickformat,
            ticksuffix=yaxis_ticksuffix,
            tickfont=dict(size=14),
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(128,128,128,0.2)'
        ),
        legend=dict(
            title='Estado',
            orientation="h",
            yanchor="bottom",
            y=-0.15,
            xanchor="center",
            x=0.5,
            font=dict(size=22)
        ),
        margin=dict(t=100, b=150, l=80, r=50),
        paper_bgcolor="#ffffff",
        plot_bgcolor='rgba(0,0,0,0)',
        height=1000,
        width=1700,
        showlegend=True
    )

    fig1.update_traces(
        text=text_labels,
        textposition="outside",
        textfont=dict(size=12, color='#333333')
    )

    fig1.show()

    # ==================== SEGUNDA GRÁFICA (Resumen por Tipo) ====================

    # Filtrar datos según el tipo seleccionado
    if tipo == 'TODO':
        datos_resumen = resumen.copy()
        titulo_resumen = "Resumen General: Sobregiro vs Ahorro por Tipo"
    else:
        datos_resumen = resumen[resumen['TIPO'] == tipo].copy()
        titulo_resumen = f"Resumen {tipo}: Sobregiro vs Ahorro"

    # Calcular totales por tipo
    resumen_por_tipo = datos_resumen.groupby('TIPO').agg({
        'COST TO DATE': 'sum',
        'BUDGET': 'sum'
    }).reset_index()

    # Calcular sobregiro y ahorro por tipo
    sobregiro_por_tipo = []
    ahorro_por_tipo = []
    tipos_labels = []

    for _, row_tipo in resumen_por_tipo.iterrows():
        tipo_actual = row_tipo['TIPO']
        presupuesto_tipo = row_tipo['BUDGET']

        # Filtrar por tipo actual
        datos_tipo = datos_resumen[datos_resumen['TIPO'] == tipo_actual]

        # Calcular sobregiro (cuentas que gastaron más)
        sobregiro_cuentas = datos_tipo[datos_tipo['ESTADO'] == 'Gastó más que el presupuesto']
        sobregiro_total = (sobregiro_cuentas['COST TO DATE'] - sobregiro_cuentas['BUDGET']).sum()
        porcentaje_sobregiro = (sobregiro_total / presupuesto_tipo) * 100 if presupuesto_tipo > 0 else 0

        # Calcular ahorro (cuentas que gastaron menos)
        ahorro_cuentas = datos_tipo[datos_tipo['ESTADO'] == 'Ahorro']
        ahorro_total = (ahorro_cuentas['BUDGET'] - ahorro_cuentas['COST TO DATE']).sum()
        porcentaje_ahorro = (ahorro_total / presupuesto_tipo) * 100 if presupuesto_tipo > 0 else 0

        sobregiro_por_tipo.append(porcentaje_sobregiro)
        ahorro_por_tipo.append(porcentaje_ahorro)
        tipos_labels.append(tipo_actual)

    # Crear la segunda gráfica
    fig2 = go.Figure()

    # Agregar barra para el porcentaje de sobregiro
    fig2.add_trace(go.Bar(
        x=tipos_labels,
        y=sobregiro_por_tipo,
        name='Porcentaje Sobregiro',
        marker_color="#DB6A56",
        text=[f"{val:.1f}%" for val in sobregiro_por_tipo],
        textposition="outside",
        textfont=dict(size=12, color='#333333')
    ))

    # Agregar barra para el porcentaje de ahorro
    fig2.add_trace(go.Bar(
        x=tipos_labels,
        y=ahorro_por_tipo,
        name='Porcentaje Ahorro',
        marker_color="#60D3B4",
        text=[f"{val:.1f}%" for val in ahorro_por_tipo],
        textposition="outside",
        textfont=dict(size=12, color='#333333')
    ))

    # Personalizar el diseño de la segunda gráfica
    fig2.update_layout(
        title={
            'text': titulo_resumen,
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 24, 'color': '#333333', 'family': 'Berlin Sans FB'}
        },
        xaxis=dict(
            title='Tipos',
            title_font=dict(size=20),
            tickfont=dict(size=14)
        ),
        yaxis=dict(
            title='Porcentaje (%)',
            title_font=dict(size=20),
            tickformat='.2f',
            ticksuffix='%',
            tickfont=dict(size=14),
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(128,128,128,0.2)'
        ),
        legend=dict(
            title='Indicadores',
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5,
            font=dict(size=27)
        ),
        margin=dict(t=100, b=100, l=50, r=50),
        paper_bgcolor="#ffffff",
        plot_bgcolor='rgba(0,0,0,0)',
        height=700,
        width=1700,
        barmode='group'
    )

    fig2.show()


# Figura 4 -  Top 15 Cuentas
# Figura 5 - Resumen General

In [86]:
widgets.interact(mostrar_graficas, estado=selector_estado, tipo=selector_tipo)

interactive(children=(Dropdown(description='Estado:', layout=Layout(width='300px'), options=('Gastó más que el…

<function __main__.mostrar_graficas(estado, tipo)>